In [1]:
import torch
from torch import nn
from torchinfo import summary

In [2]:
def conv1x1(inplanes, out_planes, stride=1):
    return nn.Conv2d(inplanes, out_planes, kernel_size=1, stride=stride, bias=False)

In [3]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.layers = nn.Sequential(*[
            conv3x3(inplanes, planes, stride),
            norm_layer(planes),
            nn.ReLU(inplace=True),
            conv3x3(planes, planes),
            norm_layer(planes),
        ])
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.layers(x)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [5]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        self.layers = nn.Sequential(*[
            conv1x1(inplanes, width),
            norm_layer(width),
            nn.ReLU(inplace=True),
            conv3x3(width, width, stride, groups, dilation),
            norm_layer(width),
            nn.ReLU(inplace=True),
            conv1x1(width, planes * self.expansion),
            norm_layer(planes * self.expansion),
        ])
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.layers(x)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)

        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, block=BasicBlock, n_classes=1000, depths=[2, 2, 2, 2]):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.inplanes),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.layers = nn.Sequential(
            self._make_layer(block, depths[0], 64),
            self._make_layer(block, depths[1], 128, stride=2),
            self._make_layer(block, depths[2], 256, stride=2),
            self._make_layer(block, depths[3], 512, stride=2),   
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Linear(512 * block.expansion, n_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.layers(x)
        x = self.avgpool(x)
        x = x.flatten(1)
        x = self.head(x)
        return x
    
    def _make_layer(self, block, depth, planes, stride=1):
        dw = None
        if stride !=1 or self.inplanes != planes * block.expansion:
            dw = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride=stride),
                nn.BatchNorm2d(planes * block.expansion)
            )
        layers = [
            block(self.inplanes, planes, stride=stride, downsample=dw)
        ]
        self.inplanes = planes * block.expansion
        for _ in range(1, depth):
            layers.append(
                block(self.inplanes, planes)
            )
        return nn.Sequential(*layers)
        

In [7]:
def resnet18(n_classes=1000):
    return ResNet(block=BasicBlock, n_classes=n_classes, depths=[2, 2, 2, 2])
def resnet34(n_classes=1000):
    return ResNet(block=BasicBlock, n_classes=n_classes, depths=[3, 4, 6, 3])
def resnet50(n_classes=1000):
    return ResNet(block=Bottleneck, n_classes=n_classes, depths=[3, 4, 6, 3])
def resnet101(n_classes=1000):
    return ResNet(block=Bottleneck, n_classes=n_classes, depths=[3, 4, 23, 3])

In [8]:
r18 = resnet18()
r34 = resnet34()
r50 = resnet50()
r101 = resnet101()

In [9]:
models = [r18, r34, r50, r101]

In [10]:
%%time
inp = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    r18(inp).shape

CPU times: user 56.9 ms, sys: 31.2 ms, total: 88.1 ms
Wall time: 80.3 ms


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [11]:
%%time
inp = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    r101(inp).shape

CPU times: user 197 ms, sys: 50.3 ms, total: 247 ms
Wall time: 128 ms


In [14]:
summary(r101, (1, 3, 224, 224), depth=1)

Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        --                        --
├─Sequential: 1-1                             [1, 64, 56, 56]           9,536
├─Sequential: 1-2                             [1, 2048, 7, 7]           42,490,624
├─AdaptiveAvgPool2d: 1-3                      [1, 2048, 1, 1]           --
├─Linear: 1-4                                 [1, 1000]                 2,049,000
Total params: 44,549,160
Trainable params: 44,549,160
Non-trainable params: 0
Total mult-adds (G): 7.80
Input size (MB): 0.60
Forward/backward pass size (MB): 259.72
Params size (MB): 178.20
Estimated Total Size (MB): 438.52

In [13]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [14]:
for m in models:
    print_size_of_model(m)

Size (MB): 46.836045
Size (MB): 87.330125
Size (MB): 102.545153
Size (MB): 178.823773


In [15]:
def fmat(n):
    return "{:.2f}M".format(n / 1e6)

In [16]:
def params(model, f=True):
    s = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return fmat(s) if f else s

In [17]:
for m in models:
    print(params(m))

11.69M
21.80M
25.56M
44.55M
